# Human in the loop

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

## Allow messages to be overwritten

In [3]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [4]:
tool = TavilySearchResults(max_results=2)

/var/folders/ks/mc5n5p95489ggm_wj27wwpyr0000gn/T/ipykernel_27274/4289725543.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


# Human in the loop approval

In [5]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-5-nano")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [7]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='d527051b-0483-4197-9e3a-01f89342d881'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nPP0gFC32oIVUh6qerlnI8IY', 'function': {'arguments': '{"query":"San Francisco CA weather current"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 234, 'total_tokens': 585, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3dp7ri7Lg9cQn2P77JHn5up4USLG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d00fd6b3-ae36-4347-b6e9-ced728f13a55-0', tool_calls=[{'name': 'tavily_search_results_js

The flow was interrupted after the tool call, lets inspect the state:

In [8]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='d527051b-0483-4197-9e3a-01f89342d881'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nPP0gFC32oIVUh6qerlnI8IY', 'function': {'arguments': '{"query":"San Francisco CA weather current"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 234, 'total_tokens': 585, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3dp7ri7Lg9cQn2P77JHn5up4USLG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d00fd6b3-ae36-4347-b6e9-ced728f13a55-0', tool_calls=[{'name': 'tav

In [10]:
abot.graph.get_state(thread).next

('action',)

We can see the next state is to call the action, we can carry on by calling the stream with none instead of messsages:

In [11]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco CA weather current'}, 'id': 'call_nPP0gFC32oIVUh6qerlnI8IY', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="[{'title': 'San Francisco weather in August 2025', 'url': 'https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=August', 'content': '| Month | Temperatures | Weather | Rainy Days | Dry Days | Snowy Days | Rainfall |\\n| --- | --- | --- | --- | --- | --- | --- |\\n| January | 15° / 7° | Good | 4 | 27 | 0 | 63 mm |\\n| February | 16° / 7° | Good | 4 | 24 | 0 | 61 mm |\\n| March | 17° / 8° | Good | 5 | 26 | 0 | 62 mm |\\n| April | 18° / 9° | Good | 2 | 28 | 0 | 22 mm |\\n| May | 20° / 10° | Good | 1 | 30 | 0 | 13 mm |\\n| June | 23° / 12° | Perfect | 0 | 30 | 0 | 6 mm |\\n| July | 25° / 14° | Perfect | 0 | 31 | 0 | 3 mm | [...] Partly cloudy\\nPartly cloudy\\nSunny\\nPartly cloudy\\nMist\\nSunny\\nPartly cloudy\\nSunny\\nPartly cloudy\\nSunny\

In [12]:
abot.graph.get_state(thread).next

('action',)

The agent still wants to run actions, so lets add human in the loop:

In [13]:
messages = [HumanMessage("What is the weather in Kensington London?")]
thread = {"configurable": {"thread_id": "2"}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

while abot.graph.get_state(thread).next:
    print(f"\n {abot.graph.get_state} \n")
    _input = input("Proceed?")
    if _input != 'y':
        print("terminating")
        break
    else:
        for event in abot.graph.stream(None, thread):
            for v in event.values():
                print(v)

{'messages': [HumanMessage(content='What is the weather in Kensington London?', additional_kwargs={}, response_metadata={}, id='c0ec6f5e-1256-415b-8f74-c7b89b3cb761'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LqbQR3OKjKyH2BxrUECYAKfa', 'function': {'arguments': '{"query":"Kensington London weather current"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 352, 'prompt_tokens': 236, 'total_tokens': 588, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gRcoIi7QNciTVLWdMoFwdRCsToa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--511650d9-ff71-4a63-b340-7b627d341063-0', tool_calls=[{'name': 'tavily

# Modifying the state

In [14]:
messages = [HumanMessage("What is the weather in LA?")]
thread = {"configurable": {"thread_id": "4"}}

# Run model until interrupt
for event in abot.graph.stream({"messages": messages}, thread):
    for value in event.values():
        print(value)

{'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='e16d2743-fcea-4ec6-80a5-4d1c778e72d5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'function': {'arguments': '{"query":"Los Angeles weather right now"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 235, 'total_tokens': 586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gZEF2hBRvOVmqZgj0IlQxiuZ9NL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8c27efce-9cd0-466a-ae78-ab5dbce19edb-0', tool_calls=[{'name': 'tavily_search_results_jso

In [16]:
current_value = abot.graph.get_state(thread)

Lets view the tool calls in this snapshot of the state

In [21]:
current_value.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'London England weather right now'},
  'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla',
  'type': 'tool_call'}]

Lets update the tool call

In [18]:
current_value.values["messages"][-1].tool_calls = [{
    'name': 'tavily_search_results_json',
    'args': {'query': 'London England weather right now'},
    'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla',
    'type': 'tool_call'
}]

Now we need to update the state of the graph after making this change

In [22]:
abot.graph.update_state(thread, current_value.values)

{'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='e16d2743-fcea-4ec6-80a5-4d1c778e72d5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'function': {'arguments': '{"query":"Los Angeles weather right now"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 235, 'total_tokens': 586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gZEF2hBRvOVmqZgj0IlQxiuZ9NL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8c27efce-9cd0-466a-ae78-ab5dbce19edb-0', tool_calls=[{'name': 'tavily_search_results_jso

{'configurable': {'thread_id': '4',
  'checkpoint_ns': '',
  'checkpoint_id': '1f077687-8b6a-643c-8003-9b86169aa664'}}

In [23]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='e16d2743-fcea-4ec6-80a5-4d1c778e72d5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'function': {'arguments': '{"query":"Los Angeles weather right now"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 235, 'total_tokens': 586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gZEF2hBRvOVmqZgj0IlQxiuZ9NL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8c27efce-9cd0-466a-ae78-ab5dbce19edb-0', tool_calls=[{'name': 'tavi

Now we can carry on execution:

In [24]:
for event in abot.graph.stream(None, thread):
    for value in event.values():
        print(value)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'London England weather right now'}, 'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="[{'title': 'London, London, United Kingdom Monthly Weather', 'url': 'https://www.accuweather.com/en/gb/london/ec4a-2/august-weather/328328', 'content': 'Get the monthly weather forecast for London, London, United Kingdom, including daily high/low, historical averages, to help you plan ahead.', 'score': 0.98531}, {'title': 'London weather in August 2025', 'url': 'https://www.weather25.com/europe/united-kingdom/london?page=month&month=August', 'content': 'The temperatures in London in August are comfortable with low of 57°F and and high up to 69°F. You can expect about 3 to 8 days of rain in London during the', 'score': 0.98236}]", name='tavily_search_results_json', id='f0c0276c-196e-4f0c-b028-4bd373445ef4', tool_call_id='call_cn9Y0zDBB1clF5UspZt7Wzla')]}
{'messages': [Human

# Time travel

In [43]:
# Save list of prev states in a list

states = []
for state in abot.graph.get_state_history(thread):
    print(state, end="\n---\n")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='13c5b4a7-751d-4a8b-a2a6-e2e302bf8f16'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Py62CGJRg3Pjiv9IYnZAKp1b', 'function': {'arguments': '{"query":"current weather Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 235, 'total_tokens': 393, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gpU8YkItthS0LnK4T9Pl4Te2t0q', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d241c523-20ae-46b7-8655-81a3f48340ce-0', tool_calls=[{'name': 'tavily

In [26]:
states[-1]

StateSnapshot(values={'messages': []}, next=('__start__',), config={'configurable': {'thread_id': '4', 'checkpoint_ns': '', 'checkpoint_id': '1f077676-973d-6fea-bfff-7056fdea0e25'}}, metadata={'source': 'input', 'step': -1, 'parents': {}}, created_at='2025-08-12T10:30:48.302380+00:00', parent_config=None, tasks=(PregelTask(id='ae513621-9479-24fe-29ed-e62d0b3aeb10', name='__start__', path=('__pregel_pull', '__start__'), error=None, interrupts=(), state=None, result={'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='e16d2743-fcea-4ec6-80a5-4d1c778e72d5')]}),), interrupts=())

In [30]:
# How to resume from prev state:
for event in abot.graph.stream(None, states[-1].config):
    for value in event.values():
        print(value)

{'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='13c5b4a7-751d-4a8b-a2a6-e2e302bf8f16'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Py62CGJRg3Pjiv9IYnZAKp1b', 'function': {'arguments': '{"query":"current weather Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 235, 'total_tokens': 393, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gpU8YkItthS0LnK4T9Pl4Te2t0q', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d241c523-20ae-46b7-8655-81a3f48340ce-0', tool_calls=[{'name': 'tavily_search_results_json'

# Go back in time to edit history

In [47]:
to_replay = states[-3]
print(to_replay)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='e16d2743-fcea-4ec6-80a5-4d1c778e72d5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'function': {'arguments': '{"query":"Los Angeles weather right now"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 235, 'total_tokens': 586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gZEF2hBRvOVmqZgj0IlQxiuZ9NL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8c27efce-9cd0-466a-ae78-ab5dbce19edb-0', tool_calls=[{'name': 'tavi

In [48]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [49]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='e16d2743-fcea-4ec6-80a5-4d1c778e72d5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'function': {'arguments': '{"query":"Los Angeles weather right now"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 235, 'total_tokens': 586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gZEF2hBRvOVmqZgj0IlQxiuZ9NL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8c27efce-9cd0-466a-ae78-ab5dbce19edb-0', tool_calls=[{'name': 'tavily_search_results_jso

In [50]:
for event in abot.graph.stream(None, branch_state):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="[{'title': 'Los Angeles, CA Monthly Weather - AccuWeather', 'url': 'https://www.accuweather.com/en/us/los-angeles/90012/august-weather/347625', 'content': 'Get the monthly weather forecast for Los Angeles, CA, including daily high/low, historical averages, to help you plan ahead.', 'score': 0.98598}, {'title': 'Weather Prediction Center (WPC) Home Page - NOAA', 'url': 'https://www.noaa.gov/weather-prediction-center', 'content': '12-Hourly QPF (Day 1-3) 6-Hourly QPF (Day 1-3) 48-Hour QPF (Day 4-5/6-7) ... Valid 01Z Tue Aug 12 2025 - 12Z Tue Aug 12 2025. Valid 12Z Tue Aug 12 2025', 'score': 0.98445}]", name='tavily_search_results_json', id='e5687e94-e8d2-4b51-8aef-5d3d75b2f3cc', tool_call_id='call_cn9Y0zDBB1clF5UspZt7Wzla')]}
{'messages': [HumanMessage(content='What is the

# Append messages to a given state

In [51]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='e16d2743-fcea-4ec6-80a5-4d1c778e72d5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'function': {'arguments': '{"query":"Los Angeles weather right now"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 235, 'total_tokens': 586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gZEF2hBRvOVmqZgj0IlQxiuZ9NL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8c27efce-9cd0-466a-ae78-ab5dbce19edb-0', tool_calls=[{'name': 'tavi

In [52]:
# Create mock response to action

state_update = state_update = {"messages": [ToolMessage(
    tool_call_id=to_replay.values['messages'][-1].tool_calls[0]['id'],
    name="tavily_search_results_json",
    content="9000 degree celcius",
)]}

In [53]:
branch_to_add = abot.graph.update_state(to_replay.config, state_update, as_node="action")

# This updates the graph as if the output came from the "action" node

In [54]:
for event in abot.graph.stream(None, branch_to_add):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='What is the weather in LA?', additional_kwargs={}, response_metadata={}, id='e16d2743-fcea-4ec6-80a5-4d1c778e72d5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cn9Y0zDBB1clF5UspZt7Wzla', 'function': {'arguments': '{"query":"Los Angeles weather right now"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 351, 'prompt_tokens': 235, 'total_tokens': 586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3gZEF2hBRvOVmqZgj0IlQxiuZ9NL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8c27efce-9cd0-466a-ae78-ab5dbce19edb-0', tool_calls=[{'name': 'tavily_search_results_jso